In [2]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt

In [3]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [4]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [5]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [6]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv'

In [13]:
contagiados_comunal=pd.read_csv(url)
contagiados_comunal['Fecha']=pd.to_datetime(contagiados_comunal['Fecha'],errors='coerce').dt.to_pydatetime()

In [14]:
for k in ['@ivanMSC']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema @ivanMSC exists, will not be created


In [15]:
name='contagios_comunal@ivanMSC'
schema='@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
contagiados_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
contagiados_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table contagios_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/contagios_comunal@ivanMSC_25082020_075624.csv in cache


In [17]:
contagiados_comunal.Region.unique()

array(['AP', 'TA', 'AN', 'AT', 'CO', 'VS', 'RM', 'LI', 'ML', 'NB', 'BI',
       'AR', 'LR', 'LL', 'AI', 'MA'], dtype=object)

In [20]:
contagiados_comunal['Region']=contagiados_comunal['Region'].replace({'AP':'Arica y Parinacota', 'TA':'Tarapacá', 'AN':'Antofagasta', 'AT':'Atacama', 
                                       'CO':'Coquimbo', 'VS':'Valparaíso', 'RM':'Región Metropolitana', 'LI':"OHiggins", 'ML':'Maule', 'NB':'Ñuble', 
                                       'BI':'Bío-Bío',
       'AR':'Araucanía', 'LR':'Los Ríos', 'LL':'Los Lagos', 'AI':'Aysén', 'MA':'Magallanes'})